In [16]:
import json
from tqdm import tqdm

import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch

import faiss

In [15]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru").to(device)

In [17]:
with open("../naked_science_corpus.json", "r") as f:
    data = json.load(f)

corpus_titles = np.array([article["article_title"] for article in data])

batch_size = 32
corpus_embeddings = []
batches = np.array_split(
    corpus_titles,
    np.floor(len(corpus_titles) / batch_size),
)

for sentences_batch in tqdm(batches):
    sentences_batch = list(sentences_batch)
    features = tokenizer(
        sentences_batch,
        max_length=48,
        padding=True,
        truncation=True,
        return_tensors="pt",
    ).to(device)
    with torch.no_grad():
        embeddings = model(**features)
        embeddings = (
            mean_pooling(embeddings, features["attention_mask"]).detach().cpu().numpy()
        )
        corpus_embeddings.extend(embeddings)
corpus_embeddings = np.array(corpus_embeddings)
faiss.normalize_L2(corpus_embeddings)

100%|██████████| 125/125 [00:02<00:00, 46.57it/s]


In [21]:
index_cache_path = "index.bin"
embedding_dim = 1024
index = faiss.IndexFlatL2(embedding_dim)


index.train(corpus_embeddings)
index.add(corpus_embeddings)

print("Saving index to:", index_cache_path)
faiss.write_index(index, index_cache_path)

Saving index to: index.bin


In [39]:
query = "атомная физика"
k = 10

features = tokenizer(
    [query],
    max_length=48,
    padding=True,
    truncation=True,
    return_tensors="pt",
).to(device)

with torch.no_grad():
    query_embedding = model(**features)
    query_embedding = (
        mean_pooling(query_embedding, features["attention_mask"]).detach().cpu().numpy()
    )

faiss.normalize_L2(query_embedding)
distances, corpus_ids = index.search(query_embedding.reshape(1, -1), k)
hits = [
    {"corpus_id": id, "score": 1 - score}
    for id, score in zip(corpus_ids[0], distances[0])
]

for i in range(k):
    predicted_article = corpus_titles[hits[i]["corpus_id"]]
    print(f"Article: {predicted_article}")

Article: Радиохимия: путем ядерных превращений
Article: Ядерная и высокотехнологичная: медицина и атом
Article: Физики расширили понимание магнитных вихрей
Article: Физики изучили строение фотопротеинов
Article: Выведена формула биоминерала меди мулуита
Article: Математика поможет усовершенствовать глубокую переработку нефти
Article: Математика поможет предотвратить инсульт
Article: В ТюмГУ представили математическую модель разделения газов и жидкостей
Article: Найден мостик перехода от электроники к фотонике
Article: Ученые переизобрели лампу накаливания
